In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
# Carregar os dados
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# Unir as tabelas pelo movieId
merged_data = pd.merge(ratings, movies, on='movieId')


In [6]:
merged_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
# Criar a matriz de utilidade
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [8]:
# Calcular a similaridade entre os usuários
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

In [9]:
def recommend_movies(user_id, user_movie_matrix, user_similarity_df, movies, num_recommendations=5):
    # Selecionar os usuários mais similares
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]

    # Somar as avaliações ponderadas dos usuários similares
    weighted_ratings = pd.Series(0, index=user_movie_matrix.columns)
    for similar_user in similar_users:
        weighted_ratings += user_similarity_df.loc[user_id, similar_user] * user_movie_matrix.loc[similar_user]

    # Remover filmes já avaliados pelo usuário
    already_rated = user_movie_matrix.loc[user_id]
    recommendations = weighted_ratings[already_rated == 0].sort_values(ascending=False)

    # Retornar os filmes recomendados
    recommended_movies = movies[movies['movieId'].isin(recommendations.index[:num_recommendations])]
    return recommended_movies

# Exemplo de recomendação para o usuário com ID 1
recommend_movies(1, user_movie_matrix, user_similarity_df, movies)

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
659,858,"Godfather, The (1972)",Crime|Drama
2078,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery
3638,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


In [10]:
# Exemplo de recomendação para um usuário específico
recommended_movies = recommend_movies(1, user_movie_matrix, user_similarity_df, movies)
print(recommended_movies)

      movieId                                              title  \
277       318                   Shawshank Redemption, The (1994)   
507       589                  Terminator 2: Judgment Day (1991)   
659       858                              Godfather, The (1972)   
2078     2762                            Sixth Sense, The (1999)   
3638     4993  Lord of the Rings: The Fellowship of the Ring,...   

                    genres  
277            Crime|Drama  
507          Action|Sci-Fi  
659            Crime|Drama  
2078  Drama|Horror|Mystery  
3638     Adventure|Fantasy  


In [12]:

# Calcular a média de avaliação e o número de avaliações para cada filme
movie_stats = merged_data.groupby('movieId').agg({
    'rating': ['mean', 'count']
}).reset_index()

# Renomear as colunas
movie_stats.columns = ['movieId', 'mean_rating', 'rating_count']

# Unir as estatísticas com os dados dos filmes
movie_stats = pd.merge(movie_stats, movies, on='movieId')

# Filtrar filmes com pelo menos 50 avaliações
popular_movies = movie_stats[movie_stats['rating_count'] >= 50]

# Ordenar os filmes pela média de avaliação
top_movies = popular_movies.sort_values(by='mean_rating', ascending=False)

# Selecionar os top N filmes
def recommend_top_movies(top_n=5):
    return top_movies.head(top_n)

# Exemplo de recomendação dos top 5 filmes
recommended_movies = recommend_top_movies(5)
print(recommended_movies[['title', 'mean_rating', 'rating_count']])

                                                  title  mean_rating  \
277                    Shawshank Redemption, The (1994)     4.429022   
659                               Godfather, The (1972)     4.289062   
2224                                  Fight Club (1999)     4.272936   
974                               Cool Hand Luke (1967)     4.271930   
602   Dr. Strangelove or: How I Learned to Stop Worr...     4.268041   

      rating_count  
277            317  
659            192  
2224           218  
974             57  
602             97  


In [14]:
# Calcular a média de avaliação e o número de avaliações para cada filme
movie_stats = merged_data.groupby('movieId').agg({
    'rating': ['mean', 'count']
}).reset_index()

# Renomear as colunas
movie_stats.columns = ['movieId', 'mean_rating', 'rating_count']

# Unir as estatísticas com os dados dos filmes
movie_stats = pd.merge(movie_stats, movies, on='movieId')

# Filtrar filmes com pelo menos 50 avaliações
popular_movies = movie_stats[movie_stats['rating_count'] >= 50]

# Ordenar os filmes pela média de avaliação
top_movies = popular_movies.sort_values(by='mean_rating', ascending=False)

# Selecionar os top N filmes
def recommend_top_movies(top_n=5):
    return top_movies.head(top_n)

# Exemplo de recomendação dos top 5 filmes
recommended_movies = recommend_top_movies(5)
print(recommended_movies[['title', 'genres', 'mean_rating', 'rating_count']])

                                                  title  \
277                    Shawshank Redemption, The (1994)   
659                               Godfather, The (1972)   
2224                                  Fight Club (1999)   
974                               Cool Hand Luke (1967)   
602   Dr. Strangelove or: How I Learned to Stop Worr...   

                           genres  mean_rating  rating_count  
277                   Crime|Drama     4.429022           317  
659                   Crime|Drama     4.289062           192  
2224  Action|Crime|Drama|Thriller     4.272936           218  
974                         Drama     4.271930            57  
602                    Comedy|War     4.268041            97  


In [15]:
# Alterar a ordem das colunas
merged_data = merged_data[['title', 'movieId', 'genres', 'userId', 'rating', 'timestamp']]

# Salvar o DataFrame em um arquivo CSV
merged_data.to_csv('merged_data_ordered.csv', index=False)

In [18]:

from sklearn.neighbors import NearestNeighbors
import numpy as np
# Criar a matriz de utilidade
user_movie_matrix = merged_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Treinar o modelo KNN
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_movie_matrix.T)  # Transpor para ter filmes como linhas e usuários como colunas

# Função para recomendar filmes similares
def recommend_similar_movies(movie_id, n_neighbors=5):
    movie_index = user_movie_matrix.columns.get_loc(movie_id)
    distances, indices = knn_model.kneighbors(user_movie_matrix.T.iloc[movie_index, :].values.reshape(1, -1), n_neighbors=n_neighbors+1)
    
    similar_movies = []
    for i in range(1, len(distances.flatten())):  # Ignorar o próprio filme
        similar_movie_id = user_movie_matrix.columns[indices.flatten()[i]]
        similar_movie_title = movies[movies['movieId'] == similar_movie_id]['title'].values[0]
        similar_movies.append((similar_movie_id, similar_movie_title))
    
    return similar_movies

# Exemplo de recomendação para um filme específico (por exemplo, movieId = 1)
recommended_movies = recommend_similar_movies(1, 5)
print(recommended_movies)

[(3114, 'Toy Story 2 (1999)'), (480, 'Jurassic Park (1993)'), (780, 'Independence Day (a.k.a. ID4) (1996)'), (260, 'Star Wars: Episode IV - A New Hope (1977)'), (356, 'Forrest Gump (1994)')]


In [19]:
# Unir as tabelas pelo movieId
merged_data = pd.merge(ratings, movies, on='movieId')

# Criar a matriz de utilidade
user_movie_matrix = merged_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Treinar o modelo KNN
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_movie_matrix.T)  # Transpor para ter filmes como linhas e usuários como colunas

# Calcular a média de avaliação e o número de avaliações para cada filme
movie_stats = merged_data.groupby('movieId').agg({
    'rating': ['mean', 'count']
}).reset_index()

# Renomear as colunas
movie_stats.columns = ['movieId', 'mean_rating', 'rating_count']

# Filtrar filmes com pelo menos 50 avaliações
popular_movies = movie_stats[movie_stats['rating_count'] >= 50]

# Função para recomendar filmes para um usuário específico
def recommend_movies_for_user(user_id, top_n=10):
    # Filmes que o usuário já assistiu
    watched_movies = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id] > 0].index.tolist()
    
    # Filmes que o usuário ainda não assistiu
    not_watched_movies = [movie for movie in user_movie_matrix.columns if movie not in watched_movies]
    
    # Filtrar os filmes não assistidos na lista de filmes populares
    not_watched_popular_movies = popular_movies[popular_movies['movieId'].isin(not_watched_movies)]
    
    # Ordenar os filmes populares não assistidos pela média de avaliação
    top_recommendations = not_watched_popular_movies.sort_values(by='mean_rating', ascending=False).head(top_n)
    
    # Adicionar títulos dos filmes
    top_recommendations = pd.merge(top_recommendations, movies[['movieId', 'title', 'genres']], on='movieId')
    
    return top_recommendations[['title', 'genres', 'mean_rating', 'rating_count']]

# Exemplo de recomendação para um usuário específico (por exemplo, userId = 1)
recommended_movies = recommend_movies_for_user(1, 10)
print(recommended_movies)

                                               title  \
0                   Shawshank Redemption, The (1994)   
1                              Godfather, The (1972)   
2                              Cool Hand Luke (1967)   
3  Dr. Strangelove or: How I Learned to Stop Worr...   
4                                 Rear Window (1954)   
5                     Godfather: Part II, The (1974)   
6                               Departed, The (2006)   
7                                  Casablanca (1942)   
8                            Dark Knight, The (2008)   
9                                   Chinatown (1974)   

                             genres  mean_rating  rating_count  
0                       Crime|Drama     4.429022           317  
1                       Crime|Drama     4.289062           192  
2                             Drama     4.271930            57  
3                        Comedy|War     4.268041            97  
4                  Mystery|Thriller     4.261905          